In [ ]:
### Check Working directory

import os
print("Working dir: ", os.getcwd())

os.chdir(os.pardir)         # change working dir to parents dir
print("Changed dir: ", os.getcwd())

In [ ]:
import pandas as pd

file_path = 'result/res_cosine_club.xlsx'
df = pd.read_excel(file_path, sheet_name='Sheet1')

sorted_df = df.sort_values(by='Cosine Similarity', ascending=False)

assigned_users = set()

sheet3 = pd.DataFrame(columns=['Group Name', 'User1', 'User2', 'User3', 'User4'])

max_top_groups = 25
max_bottom_groups = 25

top_group_count = 0
bottom_group_count = 0

for user in df['Row1'].unique():
    if top_group_count >= max_top_groups and bottom_group_count >= max_bottom_groups:
        break

    if user in assigned_users:
        continue
    
    user_df = df[df['Row1'] == user]
    
    top_3 = user_df[~user_df['Row2'].isin(assigned_users)].head(3)
    
    bottom_3 = user_df[~user_df['Row2'].isin(assigned_users)].tail(3)
    
    if len(top_3) == 3 and top_group_count < max_top_groups:
        group_name = f'Top_Group_{user}'
        group_members = [user] + top_3['Row2'].tolist() 
        
        sheet3.loc[len(sheet3)] = [group_name] + group_members
        
        assigned_users.update(group_members)
        top_group_count += 1
    
    elif len(bottom_3) == 3 and bottom_group_count < max_bottom_groups:
        group_name = f'Bottom_Group_{user}'
        group_members = [user] + bottom_3['Row2'].tolist()
        
        sheet3.loc[len(sheet3)] = [group_name] + group_members
        
        assigned_users.update(group_members)
        bottom_group_count += 1

with pd.ExcelWriter(file_path, mode='a', engine='openpyxl') as writer:
    sheet3.to_excel(writer, sheet_name='Sheet3', index=False)

print("Top_Group 25개와 Bottom_Group 25개가 Sheet3에 저장되었습니다.")

In [ ]:
sheet2 = pd.read_excel(file_path, sheet_name='Sheet2')
sheet3 = pd.read_excel(file_path, sheet_name='Sheet3')

group_mapping = {}
for index, row in sheet3.iterrows():
    for user in row[1:5]:
        user_id = int(user)
        group_name = row['Group Name']
        group_mapping[user_id] = group_name


for i in range(1, 251): 
    user_id = i + 1
    if user_id in group_mapping:
        sheet2.at[i, 'Group Assigned'] = group_mapping[user_id]

if 'Group Assigned' in sheet2.columns:

    sheet2 = sheet2[['ID', 'EXT', 'NEU', 'AGR', 'CON', 'OPN', 'Group Assigned']]

with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    sheet2.to_excel(writer, sheet_name='Sheet2', index=False)

print("그룹명이 Sheet2의 비어 있는 열에 저장되었습니다.")


In [ ]:
sheet2_df = pd.read_excel(file_path, sheet_name='Sheet2')

filtered_df = sheet2_df.dropna(subset=['Group Assigned'])

numeric_columns = ['EXT', 'NEU', 'AGR', 'CON', 'OPN']

filtered_df_numeric = filtered_df[['Group Assigned'] + numeric_columns].apply(pd.to_numeric, errors='coerce')

grouped_df = filtered_df.groupby('Group Assigned')[numeric_columns].mean()

with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    grouped_df.to_excel(writer, sheet_name='Sheet4')

print("그룹별 평균이 Sheet4에 저장되었습니다.")


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sheet2_df = pd.read_excel(file_path, sheet_name='Sheet2')
sheet4_df = pd.read_excel(file_path, sheet_name='Sheet4')

traits_columns = ['EXT', 'NEU', 'AGR', 'CON', 'OPN']

all_users_sheet2 = sheet2_df.copy()
all_users_sheet2['UserID'] = [f"User{num}" for num in range(1, len(sheet2_df) + 1)]

included_users = sheet4_df['Group Assigned'].unique()
excluded_users = [f"User{num}" for num in range(1, len(sheet2_df) + 1) if sheet2_df.iloc[num - 1]['Group Assigned'] in included_users]

grouped_sheet4 = sheet4_df.groupby('Group Assigned')[traits_columns].mean()

def euclidean_distance(row1, row2):
    return np.sqrt(np.sum((row1 - row2) ** 2))

user_numbers = all_users_sheet2['UserID']

distance_matrix = pd.DataFrame(index=user_numbers, columns=grouped_sheet4.index)

for i, user1 in enumerate(all_users_sheet2.index):
    for group in grouped_sheet4.index:
        distance_matrix.loc[user_numbers[i], group] = euclidean_distance(
            all_users_sheet2.loc[user1, traits_columns],
            grouped_sheet4.loc[group, traits_columns]
        )

filtered_distance_matrix = distance_matrix.drop(index=excluded_users, errors='ignore')

plt.figure(figsize=(12, 12)) 
sns.heatmap(filtered_distance_matrix.astype(float), cmap='coolwarm', annot=False)

plt.gca().set_aspect('equal', adjustable='box')

plt.title('Euclidean Distance Heatmap (Excluded Included Users)')
plt.xlabel('Sheet4 Groups')
plt.ylabel('Remaining Users (Excluded Included)')

plt.xticks(rotation=45, ha='right') 
plt.yticks(rotation=0)
plt.tight_layout()  
plt.show()
